# RANKING II - RANKEADOR

In [3104]:
import numpy as np
import heapq
import re
import unicodedata
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
import ast
from sklearn.externals import joblib

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\UserTBS1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



De alguna forma, la pregunta llega a través de servicios y apoyada en el parámetro, usará el modelo que le corresponda.

In [3105]:
1 #Si abrí mi Cuenta por la web, ¿Cómo puedo realizar operaciones en ella?.
2 #¿Qué es una Cuenta Sueldo bimoneda?.
3 #¿Puedo disponer de los fondos en cualquier momento?.
4 #¿Cuánto dinero necesito para abrir una Cuenta Sueldo?.
5 #¿Qué es una franquicia en mi cuenta Sueldo?.
6 #¿Mensualmente recibo estados de cuenta?.
7 #¿Puedo hacer nuevos depósitos en mi Cuenta Sueldo una vez abierta?.
8 #¿Cuántas operaciones libres tengo en Cajeros Automáticos del BBVA Continental?.
9 #¿Cuántas operaciones libres tengo por ventanilla?.
10#¿Cuánto pago al mes por mantenimiento de mi Cuenta Sueldo y mi Tarjeta de Débito?.
11#¿Los saldos de mi Cuenta Sueldo están cubiertos por el Fondo de Seguros de Depósito?.
12#¿Puedo usar mi tarjeta de débito en establecimientos comerciales que tengan terminales POS?.
#intento="¿Puedo disponer de los fondos en cualquier momento?"
#intento="¿quiero saber si es posible disponer de mis fondos en cualquier momento?"
#intento = "¿Puedo disponer de los fondos en cualquier momento?"
#intento = "¿Cuánto dinero necesito para abrir una Cuenta Sueldo?" 
intento = "cuando puedo disponer de mis fondos" 

In [3106]:
parametro = "sueldo"

# Sí parámero == sueldo
clf =joblib.load("sueldo_model_.joblib")
clf.classes_

array(['aicawbrron', 'aomono', 'casoba', 'daerramjdo', 'ddermainsrtosotr',
       'dnfocrmo', 'donoarcaso', 'drsm', 'erantr', 'erpasr',
       'erqasrparodoarsnidmacluucnratoso', 'erttvzpu', 'facaso', 'lnan',
       'ltso', 'mjdo', 'mrantrbg', 'mtroca', 'nodocasovzaa', 'noeiurso',
       'onlrcoaobact', 'onlrva', 'pointr', 'pomocasotado', 'ra', 'rt',
       'sasrantr', 'snca', 'so', 'socasocofosodo', 'su', 'tr', 'tt',
       'urtadoeocltlpo', 'vrttpn'], dtype='<U32')

#### 1. Proceso de conversión a minúsculas

In [3107]:
intento = intento.lower()

#### 2. Proceso de destrucción de tildes

In [3108]:
def normalize(preguntas):
    return ''.join(unicodedata.normalize("NFD",i)[0] for i in str(preguntas))

In [3109]:
intento = ''.join(normalize(c) for c in str(intento))

#### 3. Destrucción de símbolos

In [3110]:
intento_preg = [intento]
intento_preg
for i in range(len(intento_preg)):
    intento_preg[i] = re.sub("\W+", " ", intento_preg[i])
    intento_preg[i] = re.sub("[^\w]", " ", intento_preg[i])
    intento_preg[i] = re.sub('[ \t\n]+', ' ', intento_preg[i])
    intento_preg[i] = intento_preg[i].strip()

#### 4. Stemming y eliminación de stopwords

In [3111]:
stemmer = PorterStemmer()

for i in range(len(intento_preg)):
    palabras = nltk.word_tokenize(intento_preg[i])
    nuevaspalabras = [stemmer.stem(palabra) for palabra in intento_preg]
    intento_preg[i]=' '.join(nuevaspalabras)

intento_preg

my_list=stopwords.words('spanish')
my_list.append('si')
my_list.append('puedo')
my_list.append('cuanto')
my_list.append('cuanta')
my_list.append('estan')
my_list.append('hacer')
    
for i in range(len(intento_preg)):
    palabras = nltk.word_tokenize(intento_preg[i])
    nuevaspalabras = [palabra for palabra in palabras if palabra not in my_list]
 

In [3112]:
 nltk.word_tokenize(intento_preg[i])

['cuando', 'puedo', 'disponer', 'de', 'mis', 'fondo']

In [3113]:
[stemmer.stem(palabra) for palabra in palabras]

['cuando', 'puedo', 'dispon', 'de', 'mi', 'fondo']

In [3114]:
for i in range(len(intento_preg)):
    palabras = nltk.word_tokenize(intento_preg[i])
    nuevaspalabras = [stemmer.stem(palabra) for palabra in palabras]
    intento_preg[i]=' '.join(nuevaspalabras)

In [3115]:
intento_preg

['cuando puedo dispon de mi fondo']

#### 5. Creando el modelo TF-IDF  [PROBLEMA!!!: Necesito la lista freq_words]

##### 5.1 Invocando al objeto freq_words a partir del archivo freq_words.joblib

In [3116]:
objects_sueldo =joblib.load("objects_sueldo.joblib")
freq_words = objects_sueldo[0]
freq_words

['cuenta',
 'sueldo',
 'twitter',
 'operacion',
 'servicio',
 'tweet',
 'actualizacion',
 'tarjeta',
 'debito',
 'libr',
 'usar',
 'comercial',
 'fondo',
 'deposito',
 'necesito',
 'vez',
 'enviar',
 'seguir',
 'encontrar',
 'respuesta',
 'mensaj',
 'directo',
 'sobr',
 'termino',
 'abri',
 'web',
 'realizar',
 'bimoneda',
 'pago',
 'mantenimiento',
 'ventanilla',
 'cajero',
 'automatico',
 'bbva',
 'continent',
 'establecimiento',
 'terminal',
 'po',
 'dispon',
 'cualquier',
 'momento',
 'saldo',
 'cubierto',
 'seguro',
 'dinero',
 'abrir',
 'nuevo',
 'abierta',
 'franquicia',
 'mensualment',
 'recibo',
 'especi',
 'retweet',
 'publico',
 'imagen',
 'editar',
 'publiqu',
 'activo',
 'modo',
 'nocturno',
 'leen',
 'ver',
 'perdieron',
 'mostrar',
 'blog',
 'significa',
 'alguien',
 'persona',
 'siguiendo',
 'sigu',
 'limit',
 'seguimiento',
 'diferencia',
 'entr',
 'suspenden',
 'denunciar',
 'spam',
 'dond',
 'ma',
 'informacion',
 'queja',
 'problema',
 'relacionado',
 'derecho',
 'a

In [3117]:
# IDF Dictionary

word_idfs_test = {}
for palabra in freq_words:
    doc_count = 0
    for pregunta in intento_preg:
        if palabra in nltk.word_tokenize(pregunta):
           doc_count += 1
    word_idfs_test[palabra] = abs(np.log(len(intento_preg)/(doc_count+1)))

word_idfs_test

# TF Matrix
    
tf_matrix_test = {}
for palabra in freq_words:
    doc_tf = []
    for pregunta in intento_preg:
        frequency = 0
        for w in nltk.word_tokenize(pregunta):
            if palabra == w:
               frequency += 1
        tf_word = frequency/len(nltk.word_tokenize(pregunta))
        doc_tf.append(tf_word)
    tf_matrix_test[palabra] = doc_tf

tf_matrix_test

# Creating the Tf-Idf Model
    
tfidf_matrix_test = []
for palabra in tf_matrix_test.keys():
    tfidf_test = []
    for value in tf_matrix_test[palabra]:
        score = value * word_idfs_test[palabra]
        tfidf_test.append(score)
    tfidf_matrix_test.append(tfidf_test)   

tfidf_matrix_test
# Finishing the Tf-Tdf model
    
X_test = np.asarray(tfidf_matrix_test)
X_test = np.transpose(X_test)

###########

X_test=pd.DataFrame(X_test,columns=freq_words)

In [3118]:
X_test

,cuenta,sueldo,twitter,operacion,servicio,tweet,actualizacion,tarjeta,debito,libr,...,problema,relacionado,derecho,autor,suplantacion,identidad,marca,u,cuestion,relacionada
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3119]:
X_test["libr"]

0    0.0
Name: libr, dtype: float64

In [3120]:
word_idfs_test

{'abierta': 0.0,
 'abri': 0.0,
 'abrir': 0.0,
 'activo': 0.0,
 'actualizacion': 0.0,
 'alguien': 0.0,
 'automatico': 0.0,
 'autor': 0.0,
 'bbva': 0.0,
 'bimoneda': 0.0,
 'blog': 0.0,
 'cajero': 0.0,
 'comercial': 0.0,
 'continent': 0.0,
 'cualquier': 0.0,
 'cubierto': 0.0,
 'cuenta': 0.0,
 'cuestion': 0.0,
 'debito': 0.0,
 'denunciar': 0.0,
 'deposito': 0.0,
 'derecho': 0.0,
 'diferencia': 0.0,
 'dinero': 0.0,
 'directo': 0.0,
 'dispon': 0.6931471805599453,
 'dond': 0.0,
 'editar': 0.0,
 'encontrar': 0.0,
 'entr': 0.0,
 'enviar': 0.0,
 'especi': 0.0,
 'establecimiento': 0.0,
 'fondo': 0.6931471805599453,
 'franquicia': 0.0,
 'identidad': 0.0,
 'imagen': 0.0,
 'informacion': 0.0,
 'leen': 0.0,
 'libr': 0.0,
 'limit': 0.0,
 'ma': 0.0,
 'mantenimiento': 0.0,
 'marca': 0.0,
 'mensaj': 0.0,
 'mensualment': 0.0,
 'modo': 0.0,
 'momento': 0.0,
 'mostrar': 0.0,
 'necesito': 0.0,
 'nocturno': 0.0,
 'nuevo': 0.0,
 'operacion': 0.0,
 'pago': 0.0,
 'perdieron': 0.0,
 'persona': 0.0,
 'po': 0.0,
 '

In [3121]:
predict_int=clf.predict(X_test)
predict_int

array(['dnfocrmo'], dtype='<U32')

In [3122]:
predict_probas=clf.predict_proba(X_test) 
predict_probas

array([[0.02844523, 0.02841292, 0.02848007, 0.02842996, 0.02844064,
        0.03196887, 0.02845883, 0.02841292, 0.02846802, 0.02843186,
        0.02842305, 0.02843217, 0.02848007, 0.02843722, 0.02841292,
        0.02844136, 0.02844706, 0.02845058, 0.02845591, 0.02843929,
        0.02842574, 0.0284386 , 0.02844221, 0.02845591, 0.02844136,
        0.02841292, 0.028442  , 0.0284695 , 0.02841292, 0.02945581,
        0.02841292, 0.02850127, 0.02846162, 0.02842915, 0.02842911]])

In [3123]:
predict_probas=np.concatenate(np.round(predict_probas,4))
predict_probas


array([0.0284, 0.0284, 0.0285, 0.0284, 0.0284, 0.032 , 0.0285, 0.0284,
       0.0285, 0.0284, 0.0284, 0.0284, 0.0285, 0.0284, 0.0284, 0.0284,
       0.0284, 0.0285, 0.0285, 0.0284, 0.0284, 0.0284, 0.0284, 0.0285,
       0.0284, 0.0284, 0.0284, 0.0285, 0.0284, 0.0295, 0.0284, 0.0285,
       0.0285, 0.0284, 0.0284])

In [3124]:
order_probs=np.sort(predict_probas)
order_probs


array([0.0284, 0.0284, 0.0284, 0.0284, 0.0284, 0.0284, 0.0284, 0.0284,
       0.0284, 0.0284, 0.0284, 0.0284, 0.0284, 0.0284, 0.0284, 0.0284,
       0.0284, 0.0284, 0.0284, 0.0284, 0.0284, 0.0284, 0.0284, 0.0285,
       0.0285, 0.0285, 0.0285, 0.0285, 0.0285, 0.0285, 0.0285, 0.0285,
       0.0285, 0.0295, 0.032 ])

In [3125]:
order_probs.std()

0.0006174073507293977

In [3126]:
prob_lim=order_probs[len(order_probs)-3]
prob_lim

0.0285

In [3127]:
pos_probs=np.where(predict_probas>=prob_lim)[0]
pos_probs

array([ 2,  5,  6,  8, 12, 17, 18, 23, 27, 29, 31, 32], dtype=int64)

In [3128]:
probs=predict_probas[pos_probs]
probs

array([0.0285, 0.032 , 0.0285, 0.0285, 0.0285, 0.0285, 0.0285, 0.0285,
       0.0285, 0.0295, 0.0285, 0.0285])

In [3129]:
df_probas=pd.DataFrame({'probas':probs,'posicion':pos_probs})
df_probas

,probas,posicion
0,0.0285,2
1,0.0320,5
2,0.0285,6
3,0.0285,8
4,0.0285,12
5,0.0285,17
6,0.0285,18
7,0.0285,23
8,0.0285,27
9,0.0295,29


In [3168]:
df_probas=df_probas.sort_values(by='probas', ascending=False)
df_probas

,probas,posicion
1,0.0320,5
9,0.0295,29
0,0.0285,2
2,0.0285,6
3,0.0285,8
4,0.0285,12
5,0.0285,17
6,0.0285,18
7,0.0285,23
8,0.0285,27


In [3131]:
pos_probs_ult=df_probas['posicion']
pos_probs_ult

1      5
9     29
0      2
2      6
3      8
4     12
5     17
6     18
7     23
8     27
10    31
11    32
Name: posicion, dtype: int64

In [3132]:
pos_probs_ult.tolist()
np.asarray(pos_probs_ult)
pos_probs_ult=pos_probs_ult[:3]

In [3133]:
pos_final=np.asarray(pos_probs_ult).tolist()
pos_final

[5, 29, 2]

In [3134]:
intencion_a=np.array(clf.classes_)
intencion_a

array(['aicawbrron', 'aomono', 'casoba', 'daerramjdo', 'ddermainsrtosotr',
       'dnfocrmo', 'donoarcaso', 'drsm', 'erantr', 'erpasr',
       'erqasrparodoarsnidmacluucnratoso', 'erttvzpu', 'facaso', 'lnan',
       'ltso', 'mjdo', 'mrantrbg', 'mtroca', 'nodocasovzaa', 'noeiurso',
       'onlrcoaobact', 'onlrva', 'pointr', 'pomocasotado', 'ra', 'rt',
       'sasrantr', 'snca', 'so', 'socasocofosodo', 'su', 'tr', 'tt',
       'urtadoeocltlpo', 'vrttpn'], dtype='<U32')

In [3135]:
pala_clave=intencion_a[pos_probs_1]
pala_clave

array(['donoarcaso', 'mtroca', 'nodocasovzaa'], dtype='<U32')

In [3136]:
pala_clave_1=list(pala_clave)
pala_clave_1

['donoarcaso', 'mtroca', 'nodocasovzaa']

In [3137]:
probas=predict_probas[pos_final]
probas

array([0.032 , 0.0295, 0.0285])

In [3138]:
orde=pd.DataFrame({'palabra':pala_clave_1,'probs':probas})
orde

,palabra,probs
0,donoarcaso,0.0320
1,mtroca,0.0295
2,nodocasovzaa,0.0285


In [3139]:
orde_1=orde.sort_values(by='probs', ascending=False)
orde_1


,palabra,probs
0,donoarcaso,0.0320
1,mtroca,0.0295
2,nodocasovzaa,0.0285


In [3140]:
etiquetas=list(orde_1['palabra'])
etiquetas

['donoarcaso', 'mtroca', 'nodocasovzaa']

##### 5.2 Invocando al objeto descripcion_m a partir del archivo freq_words.joblib

In [3141]:
descripcion_m = objects_sueldo[1]

In [3142]:
descripcion_a=np.array(descripcion_m)

In [3143]:
pala_clave_descrip=descripcion_a[pos_final]

In [3144]:
descripcion_m

['Si abrí mi Cuenta por la web, ¿Cómo puedo realizar operaciones en ella?.',
 '¿Cómo activo el modo nocturno?',
 '¿Qué es una Cuenta Sueldo bimoneda?.',
 '¿Cuál es la diferencia entre una respuesta y un Mensaje Directo?',
 '¿Dónde puedo encontrar más información sobre los Términos de servicio de Twitter?',
 '¿Puedo disponer de los fondos en cualquier momento?.',
 '¿Cuánto dinero necesito para abrir una Cuenta Sueldo?.',
 '¿Cómo puedo denunciar el spam?',
 '¿Cómo puedo enviar actualizaciones a Twitter?',
 '¿Cómo puedo encontrar personas para seguir?',
 '¿Cómo puedo enviar una queja sobre problemas relacionados con los derechos de autor, la suplantación de identidad, las marcas comerciales u otras cuestiones relacionadas con los Términos de servicio?',
 '¿Puedo editar un Tweet una vez que lo publiqué?',
 '¿Qué es una franquicia en mi cuenta Sueldo?.',
 '¿Quiénes leen mis actualizaciones?',
 '¿Qué son los límites de seguimiento?',
 '¿Qué son los Mensajes Directos?',
 '¿Puedo mostrar mis a

In [3145]:
pala_clave_descrip

array(['¿Puedo disponer de los fondos en cualquier momento?.',
       '¿Los saldos de mi Cuenta Sueldo están cubiertos por el Fondo de Seguros de Depósito?.',
       '¿Qué es una Cuenta Sueldo bimoneda?.'], dtype='<U199')

In [3146]:
pala_clave_1_descrip=list(pala_clave_descrip)

In [3147]:
orde_descrip=pd.DataFrame({'descripcion':pala_clave_1_descrip,'probs':probas})
orde_1_descrip=orde_descrip.sort_values(by='probs', ascending=False)
etiquetas_descrip=list(orde_1_descrip['descripcion'])


In [3148]:
orde_1_descrip

,descripcion,probs
0,¿Puedo disponer de los fondos en cualquier mom...,0.0320
1,¿Los saldos de mi Cuenta Sueldo están cubierto...,0.0295
2,¿Qué es una Cuenta Sueldo bimoneda?.,0.0285


In [3149]:
etiquetas_descrip

['¿Puedo disponer de los fondos en cualquier momento?.',
 '¿Los saldos de mi Cuenta Sueldo están cubiertos por el Fondo de Seguros de Depósito?.',
 '¿Qué es una Cuenta Sueldo bimoneda?.']

In [3150]:
flag=0
if len(np.unique(predict_probas))==1:
    flag=1
else:
    flag=0
flag



0

### Definición de hiperparámetros al modelo de Ranking: _umbral_

El hiperparámetro tendrá como objetivo ajustar el modelo para que la respuesta sea única o tricotómica.


$\frac{1}{len(FAQ)}$ + ${umbral}$.

In [3151]:
def zscore(probas):
    sd=probas.std()
    mean=probas.mean()
    zsc=(probas-mean)/sd
    return(zsc)

In [3152]:
zscore(predict_probas)

array([-0.26377584, -0.26377584, -0.10180822, -0.26377584, -0.26377584,
        5.56705853, -0.10180822, -0.26377584, -0.10180822, -0.26377584,
       -0.26377584, -0.26377584, -0.10180822, -0.26377584, -0.26377584,
       -0.26377584, -0.26377584, -0.10180822, -0.10180822, -0.26377584,
       -0.26377584, -0.26377584, -0.26377584, -0.10180822, -0.26377584,
       -0.26377584, -0.26377584, -0.10180822, -0.26377584,  1.51786799,
       -0.26377584, -0.10180822, -0.10180822, -0.26377584, -0.26377584])

In [3153]:
import random
import scipy as sp
from scipy.stats import uniform

In [3154]:
from math import factorial
from scipy.stats import norm
from sklearn.utils import resample
from scipy import stats

In [3155]:
sd=predict_probas.std()
mean=predict_probas.mean()

In [3156]:
stats.shapiro(predict_probas)
1/35

0.02857142857142857

In [3157]:
sd

0.0006174073507293976

In [3158]:
mean

0.028562857142857143

In [3159]:
zsc=(predict_probas-mean)/sd
zsc # -2 a 2

array([-0.26377584, -0.26377584, -0.10180822, -0.26377584, -0.26377584,
        5.56705853, -0.10180822, -0.26377584, -0.10180822, -0.26377584,
       -0.26377584, -0.26377584, -0.10180822, -0.26377584, -0.26377584,
       -0.26377584, -0.26377584, -0.10180822, -0.10180822, -0.26377584,
       -0.26377584, -0.26377584, -0.26377584, -0.10180822, -0.26377584,
       -0.26377584, -0.26377584, -0.10180822, -0.26377584,  1.51786799,
       -0.26377584, -0.10180822, -0.10180822, -0.26377584, -0.26377584])

## Probabilidades de cada pregunta del FAQ exacto

1.-  3.17340639   0.40775613   
2.-  2.99460189   0.71222964   0.08093519   
3.-  3.2830604    0.12289531   
4.-  3.05485867   0.43962911   0.43962911   
5.-  2.99243639   0.69896325   0.10921301   
6.-  3.26887947   
7.-  2.98743358   0.38452115   0.38452115   0.32536405   
8.-  2.78763451   1.45940865   0.0327957    
9.-  3.14529559   0.6045702    0.14540296   
10.- 2.98872176   0.38983327   0.38983327   0.30320366   
11.- 2.98881752   0.38630975   0.30498138   0.30498138   
12.- 3.16990576   0.63757311   

In [3160]:
np.where(zsc>=2)

(array([5], dtype=int64),)

In [3161]:
1/len(descripcion_m)

0.02857142857142857

In [3162]:
sum(np.where(zsc>=2))

array([5], dtype=int64)

In [3163]:
a = [1,3,2,2,2]
sum(predict_probas)

0.9996999999999995

In [3164]:
len(np.where(zsc>=2)[0])

1

In [3165]:
if len(np.where(zsc>=2)[0])==0 or len(np.where(zsc>=2)[0])>=4:
    print("Por favor sea mas especifico")
elif len(np.where(zsc>=2)[0])>=1 or len(np.where(zsc>=2)[0])<=3:
    print("Estas son nuestras sugerencias")

Estas son nuestras sugerencias


In [3166]:
n = len(predict_probas)
reps = 10000
xb = np.random.choice(x, (n, reps))
mb = xb.mean(axis=0)
mb.sort()

np.percentile(mb, [.05, .95])

array([0.3448772 , 0.38470144])

In [3167]:
c=1/35
c+0.1*c

0.03142857142857143